In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate

In [ ]:
import torch
from transformers import AutoConfig, AutoModel, GPT2Tokenizer, TextDataset, TrainingArguments
from transformers import DataCollatorForLanguageModeling, Trainer, AutoTokenizer, GPT2LMHeadModel

In [ ]:
config = AutoConfig.from_pretrained("/content/drive/MyDrive/NLP/final_model")
model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/NLP/final_model")
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [ ]:
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path = "/content/drive/MyDrive/NLP/dataset_1.csv",
    block_size=128
)

validation_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="/content/drive/MyDrive/NLP/v.csv",
    block_size=128
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
from copy import deepcopy
import tqdm

def get_fisher_diag(model, dataset, params, empirical=True):
  fisher = {}
  params_dict = dict(params)
  for n, p in deepcopy(params_dict).items():
    p.data.zero_()
    fisher[n] = p.data.clone().detach().requires_grad_()

  model.eval()

  dataset = tqdm(dataset, total=len(dataset))

  for batch in dataset:
    input, _, _, target = batch

    input = input.to(device)
    target = target.to(device)

    model.zero_grad()
    output = model(input)
    output = output.logits
    output = output.view(-1, output.size(-1))
    # output = model(input).view(1, -1)
    if empirical:
      label = target.view(-1)
    else:
      label = torch.argmax(output, dim=1)

    cross_entropy_loss = torch.nn.functional.cross_entropy(output, label)
    cross_entropy_loss.backward()

    for n, p in model.named_parameters():
      fisher[n].data += p.grad.data ** 2 / len(dataset)

  fisher = {n: p for n, p in fisher.items()}
  return fisher

def get_ewc_loss(model, fisher, p_old):
  loss = 0
  for n, p in model.named_parameters():
    _loss = fisher[n] * (p - p_old[n]) ** 2
    loss += _loss.sum()
  return loss

# Training Loop

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True, drop_last=True)
eval_dataloader = DataLoader(validation_dataset, batch_size=4, shuffle=True, drop_last=True)

In [ ]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
%%time
from tqdm import tqdm

model.to(device)

fisher_matrix = get_fisher_diag(model, train_dataloader, model.named_parameters())
prev_params = {n: p.data.clone() for n, p in model.named_parameters()}

learning_rate = 0.001
ewc_lambda = 0.1

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for epoch in range(3):
  model.train()
  total_loss = 0.

  train_dataloader = tqdm(train_dataloader, total=len(train_dataloader))

  for batch in train_dataloader:
    input, _, _, target = batch

    input = input.to(device)
    target = target.to(device)

    optimizer.zero_grad()
    # print(target.shape)
    output = model(input)
    output = output.logits
    output = output.view(-1, output.size(-1))

    label = target.view(-1)

    # Original loss
    ce_loss = torch.nn.functional.cross_entropy(output, label)

    # EWC loss
    ewc_loss = get_ewc_loss(model, fisher_matrix, prev_params)

    loss = ce_loss + ewc_lambda * ewc_loss

    loss.backward()
    optimizer.step()

    train_dataloader.set_description(f"Epoch {epoch+1}")
    train_dataloader.set_postfix(loss=loss.item())

    total_loss += loss.item()

  # Update fisher matrix and previous parameters after each epoch
  if epoch < 2:
    fisher_matrix = get_fisher_diag(model, train_dataloader, model.named_parameters())
    prev_params = {n: p.data.clone() for n, p in model.named_parameters()}

  avg_loss = total_loss / len(train_dataloader)
  print(f"Epoch: {epoch+1}, Average Loss: {avg_loss}")

  # Validation
  model.eval()
  val_loss = 0.
  with torch.no_grad():

    eval_dataloader = tqdm(eval_dataloader, total=len(eval_dataloader))

    for batch in eval_dataloader:
      val_input, _, _, val_target = batch

      val_input = val_input.to(device)
      val_target = val_target.to(device)

      output_val = model(val_input)
      output_val = output_val.logits
      output_val = output_val.view(-1, output_val.size(-1))
      label_val = val_target.view(-1)

      eval_dataloader.set_description(f"Epoch {epoch+1}")

      val_loss += torch.nn.functional.cross_entropy(output_val, label_val).item()

  avg_val_loss = val_loss / len(eval_dataloader)
  print(f"Epoch: {epoch+1}, Validation Loss: {avg_val_loss}")

  # Save a chekpoint
  torch.save({
      'epoch': epoch,
      'model_state_dict': model.state_dict(),
      'optimizer_state_dict': optimizer.state_dict()
  }, "/content/drive/MyDrive/NLP/EWC_model/model_checkpoint.pth")

torch.save(model.state_dict(), "/content/drive/MyDrive/NLP/EWC_model/model.pth")


  0%|          | 0/20387 [00:00<?, ?it/s]


AttributeError: ignored